In [1]:
# Authors: Remi Flamary <remi.flamary@unice.fr>
#          Stanislas Chambon <stan.chambon@gmail.com>
#
# License: MIT License

# sphinx_gallery_thumbnail_number = 3
import os
from pathlib import Path
import numpy as np
from matplotlib import pyplot as plt
import ot

rng = np.random.RandomState(42)


def im2mat(img):
    """Converts and image to matrix (one pixel per line)"""
    return img.reshape((img.shape[0] * img.shape[1], img.shape[2]))


def mat2im(X, shape):
    """Converts back a matrix to an image"""
    return X.reshape(shape)


def minmax(img):
    return np.clip(img, 0, 1)

In [2]:
# Loading images
this_file = os.path.realpath('__file__')
data_path = os.path.join(Path(this_file).parent.parent.parent, 'data')

I1 = plt.imread(os.path.join(data_path, '/home/btzivan/Bureau/TER/b0.jpg')).astype(np.float64) / 256
I2 = plt.imread(os.path.join(data_path, '/home/btzivan/Bureau/TER/b1.jpg')).astype(np.float64) / 256

X1 = im2mat(I1)
X2 = im2mat(I2)

# training samples
nb = 500
idx1 = rng.randint(X1.shape[0], size=(nb,))
idx2 = rng.randint(X2.shape[0], size=(nb,))

Xs = X1[idx1, :]
Xt = X2[idx2, :]

In [3]:
# EMDTransport
ot_emd = ot.da.EMDTransport()
ot_emd.fit(Xs=Xs, Xt=Xt)
transp_Xs_emd = ot_emd.transform(Xs=X1)
Image_emd = minmax(mat2im(transp_Xs_emd, I1.shape))

# SinkhornTransport
ot_sinkhorn = ot.da.SinkhornTransport(reg_e=1e-1)
ot_sinkhorn.fit(Xs=Xs, Xt=Xt)
transp_Xs_sinkhorn = ot_sinkhorn.transform(Xs=X1)
Image_sinkhorn = minmax(mat2im(transp_Xs_sinkhorn, I1.shape))

ot_mapping_linear = ot.da.MappingTransport(
    mu=1e0, eta=1e-8, bias=True, max_iter=20, verbose=True)
ot_mapping_linear.fit(Xs=Xs, Xt=Xt)

X1tl = ot_mapping_linear.transform(Xs=X1)
Image_mapping_linear = minmax(mat2im(X1tl, I1.shape))

ot_mapping_gaussian = ot.da.MappingTransport(
    mu=1e0, eta=1e-2, sigma=1, bias=False, max_iter=10, verbose=True)
ot_mapping_gaussian.fit(Xs=Xs, Xt=Xt)

X1tn = ot_mapping_gaussian.transform(Xs=X1)  # use the estimated mapping
Image_mapping_gaussian = minmax(mat2im(X1tn, I1.shape))

It.  |Loss        |Delta loss
--------------------------------
    0|3.771924e+01|0.000000e+00
    1|3.718816e+01|-1.407977e-02
    2|3.716490e+01|-6.254721e-04
    3|3.715494e+01|-2.681171e-04
    4|3.714898e+01|-1.604732e-04
    5|3.714495e+01|-1.084680e-04
    6|3.714200e+01|-7.931984e-05
    7|3.713975e+01|-6.065162e-05
    8|3.713799e+01|-4.743247e-05
    9|3.713656e+01|-3.844413e-05
   10|3.713541e+01|-3.080998e-05
   11|3.713443e+01|-2.648793e-05
   12|3.713358e+01|-2.300952e-05
   13|3.713286e+01|-1.930977e-05
   14|3.713221e+01|-1.748961e-05
   15|3.713164e+01|-1.543008e-05
   16|3.713114e+01|-1.331205e-05
   17|3.713069e+01|-1.210226e-05
   18|3.713032e+01|-9.989650e-06
It.  |Loss        |Delta loss
--------------------------------
    0|4.241752e+01|0.000000e+00
    1|4.160839e+01|-1.907545e-02
    2|4.151685e+01|-2.199966e-03
    3|4.148976e+01|-6.525043e-04
    4|4.147822e+01|-2.781511e-04
    5|4.147177e+01|-1.556005e-04
    6|4.146753e+01|-1.020918e-04
    7|4.146460e+01

In [ ]:
plt.figure(1, figsize=(6.4, 3))
plt.subplot(1, 2, 1)
plt.imshow(I1)
plt.axis('off')
plt.title('Image 1')

plt.subplot(1, 2, 2)
plt.imshow(I2)
plt.axis('off')
plt.title('Image 2')
plt.tight_layout()

In [ ]:
plt.figure(2, figsize=(6.4, 5))

plt.subplot(1, 2, 1)
plt.scatter(Xs[:, 0], Xs[:, 2], c=Xs)
plt.axis([0, 1, 0, 1])
plt.xlabel('Red')
plt.ylabel('Blue')
plt.title('Image 1')

plt.subplot(1, 2, 2)
plt.scatter(Xt[:, 0], Xt[:, 2], c=Xt)
plt.axis([0, 1, 0, 1])
plt.xlabel('Red')
plt.ylabel('Blue')
plt.title('Image 2')
plt.tight_layout()

In [ ]:
plt.figure(2, figsize=(10, 5))

plt.subplot(2, 3, 1)
plt.imshow(I1)
plt.axis('off')
plt.title('Im. 1')

plt.subplot(2, 3, 4)
plt.imshow(I2)
plt.axis('off')
plt.title('Im. 2')

plt.subplot(2, 3, 2)
plt.imshow(Image_emd)
plt.axis('off')
plt.title('EmdTransport')

plt.subplot(2, 3, 5)
plt.imshow(Image_sinkhorn)
plt.axis('off')
plt.title('SinkhornTransport')

plt.subplot(2, 3, 3)
plt.imshow(Image_mapping_linear)
plt.axis('off')
plt.title('MappingTransport (linear)')

plt.subplot(2, 3, 6)
plt.imshow(Image_mapping_gaussian)
plt.axis('off')
plt.title('MappingTransport (gaussian)')
plt.tight_layout()

plt.show()
